In [25]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### scrapping all the data

In [26]:
response=requests.get("https://www.npci.org.in/what-we-do/upi/product-statistics")
html=BeautifulSoup(response.text,"html.parser")
tables=html.find_all("table",{"class":"table table-bordered"})
table=tables[0]
rows=table.find_all("tr")
column=data = [item for item in rows[0].text.strip("\n").split("\n") if item.strip() != '']
data=[]
for row in rows[1:]:
    data.append(row.text.strip().split("\n"))
prod_stats=pd.DataFrame(data,columns=column)    
prod_stats

,Month,No. of Banks live on UPI,Volume (in Mn),Value (in Cr.)
0,Jun-25,675,"18,395.01","24,03,930.69"
1,May-25,673,"18,677.46","25,14,297.01"
2,Apr-25,668,"17,893.42","23,94,925.87"
3,Mar-25,661,"18,301.51","24,77,221.61"
4,Feb-25,653,"16,106.19","21,96,481.69"
...,...,...,...,...
106,Aug-16,21,0.09,3.09
107,Jul-16,21,0.09,0.38
108,Jun-16,21,0.00,0.00
109,May-16,21,0.00,0.00


 ### cleaning data

In [27]:
prod_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Month                     111 non-null    object
 1   No. of Banks live on UPI  111 non-null    object
 2   Volume (in Mn)            111 non-null    object
 3   Value (in Cr.)            111 non-null    object
dtypes: object(4)
memory usage: 3.6+ KB


In [28]:
prod_stats.columns=prod_stats.columns.str.lower().str.replace(" ","_")

In [29]:
prod_stats

,month,no._of_banks_live_on_upi,volume_(in_mn),value_(in_cr.)
0,Jun-25,675,"18,395.01","24,03,930.69"
1,May-25,673,"18,677.46","25,14,297.01"
2,Apr-25,668,"17,893.42","23,94,925.87"
3,Mar-25,661,"18,301.51","24,77,221.61"
4,Feb-25,653,"16,106.19","21,96,481.69"
...,...,...,...,...
106,Aug-16,21,0.09,3.09
107,Jul-16,21,0.09,0.38
108,Jun-16,21,0.00,0.00
109,May-16,21,0.00,0.00


In [30]:
prod_stats["month"]=pd.to_datetime(prod_stats["month"],format="%b-%y")

In [31]:
prod_stats

,month,no._of_banks_live_on_upi,volume_(in_mn),value_(in_cr.)
0,2025-06-01,675,"18,395.01","24,03,930.69"
1,2025-05-01,673,"18,677.46","25,14,297.01"
2,2025-04-01,668,"17,893.42","23,94,925.87"
3,2025-03-01,661,"18,301.51","24,77,221.61"
4,2025-02-01,653,"16,106.19","21,96,481.69"
...,...,...,...,...
106,2016-08-01,21,0.09,3.09
107,2016-07-01,21,0.09,0.38
108,2016-06-01,21,0.00,0.00
109,2016-05-01,21,0.00,0.00


In [32]:
prod_stats.dtypes

month                       datetime64[ns]
no._of_banks_live_on_upi            object
volume_(in_mn)                      object
value_(in_cr.)                      object
dtype: object

In [33]:
prod_stats.rename(columns={"no._of_banks_live_on_upi":"total_banks"},inplace=True)
prod_stats.rename(columns={"volume_(in_mn)":"total_transaction"},inplace=True)
prod_stats.rename(columns={"value_(in_cr.)":"value"},inplace=True)

In [34]:
prod_stats["total_banks"]=prod_stats["total_banks"].astype(int)

In [35]:
prod_stats

,month,total_banks,total_transaction,value
0,2025-06-01,675,"18,395.01","24,03,930.69"
1,2025-05-01,673,"18,677.46","25,14,297.01"
2,2025-04-01,668,"17,893.42","23,94,925.87"
3,2025-03-01,661,"18,301.51","24,77,221.61"
4,2025-02-01,653,"16,106.19","21,96,481.69"
...,...,...,...,...
106,2016-08-01,21,0.09,3.09
107,2016-07-01,21,0.09,0.38
108,2016-06-01,21,0.00,0.00
109,2016-05-01,21,0.00,0.00


In [36]:
def mn_to_num(x):
    num=x.replace(",","")
    numb=float(num)
    return numb*1000000
    

In [37]:
prod_stats["total_transaction"] = prod_stats["total_transaction"].apply(mn_to_num)

In [38]:
prod_stats["total_transaction"]=prod_stats["total_transaction"].astype(int)

In [39]:
prod_stats

,month,total_banks,total_transaction,value
0,2025-06-01,675,18395010000,"24,03,930.69"
1,2025-05-01,673,18677460000,"25,14,297.01"
2,2025-04-01,668,17893420000,"23,94,925.87"
3,2025-03-01,661,18301510000,"24,77,221.61"
4,2025-02-01,653,16106190000,"21,96,481.69"
...,...,...,...,...
106,2016-08-01,21,90000,3.09
107,2016-07-01,21,90000,0.38
108,2016-06-01,21,0,0.00
109,2016-05-01,21,0,0.00


In [40]:
def cr_to_num(x):
    num=x.replace(",","")
    numb=float(num)
    return numb*10000000

In [41]:
prod_stats["value"] = prod_stats["value"].apply(cr_to_num)

In [42]:
prod_stats["value"]=prod_stats["value"].astype(int)

In [43]:
prod_stats

,month,total_banks,total_transaction,value
0,2025-06-01,675,18395010000,24039306900000
1,2025-05-01,673,18677460000,25142970099999
2,2025-04-01,668,17893420000,23949258700000
3,2025-03-01,661,18301510000,24772216100000
4,2025-02-01,653,16106190000,21964816900000
...,...,...,...,...
106,2016-08-01,21,90000,30900000
107,2016-07-01,21,90000,3800000
108,2016-06-01,21,0,0
109,2016-05-01,21,0,0


In [44]:
prod_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   month              111 non-null    datetime64[ns]
 1   total_banks        111 non-null    int64         
 2   total_transaction  111 non-null    int64         
 3   value              111 non-null    int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 3.6 KB


In [45]:
prod_stats


,month,total_banks,total_transaction,value
0,2025-06-01,675,18395010000,24039306900000
1,2025-05-01,673,18677460000,25142970099999
2,2025-04-01,668,17893420000,23949258700000
3,2025-03-01,661,18301510000,24772216100000
4,2025-02-01,653,16106190000,21964816900000
...,...,...,...,...
106,2016-08-01,21,90000,30900000
107,2016-07-01,21,90000,3800000
108,2016-06-01,21,0,0
109,2016-05-01,21,0,0


In [182]:
prod_stats.to_excel("C:\\Users\\shrut\\OneDrive\\Desktop\\python practice\\pro_stats.xlsx", index=False)


### load all data

In [245]:
upi_apps=pd.read_excel("C:\\Users\\shrut\\OneDrive\\Desktop\\upi apps cleaned data.xlsx")
chargeback=pd.read_excel("C:\\Users\\shrut\\OneDrive\\Desktop\\chargeback celaned data.xlsx")
p2p=pd.read_excel("C:\\Users\\shrut\\OneDrive\\Desktop\\p2p p2m cleaned.xlsx")

### cleaning the data

### upi apps 


In [249]:
upi_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2905 entries, 0 to 2904
Data columns (total 12 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   Application Name                               2905 non-null   object        
 1   date                                           2905 non-null   datetime64[ns]
 2   Customer Initiated Transactions(volume in mn)  2905 non-null   float64       
 3   Customer Initiated Transactions(value in cr)   2905 non-null   float64       
 4   B2C Transactions(volume in mn)                 2904 non-null   float64       
 5   B2C Transactions(value in cr)                  2904 non-null   float64       
 6   B2B Transactions( volumne in mn)               2905 non-null   float64       
 7   B2B Transactions( value in cr)                 2905 non-null   float64       
 8   On-us Transactions(volume in mn)               2769 non-nu

In [250]:
# conerting to million and crore


In [261]:
def volume_to_num(x):
    return x * 1_000_000

In [252]:
upi_apps.columns=upi_apps.columns.str.lower().str.replace(" ","_")

In [253]:
upi_apps

,application_name,date,customer_initiated_transactions(volume_in_mn),customer_initiated_transactions(value_in_cr),b2c_transactions(volume_in_mn),b2c_transactions(value_in_cr),b2b_transactions(_volumne_in_mn),b2b_transactions(_value_in_cr),on-us_transactions(volume_in_mn),on-us_transactions(value_in_cr),total(volume_in_mn),total(value_in_cr)
0,Airtel Payments Bank Apps,2022-04-01,5.29,454.64,0.00,0.00,0.00,0.00,0.01,6.26,5.30,460.90
1,Amazon Pay,2022-04-01,73.21,6699.57,0.00,0.00,0.00,0.00,0.00,0.00,73.21,6699.57
2,AU Small Finance Bank Apps,2022-04-01,0.32,109.89,0.00,0.00,0.00,0.00,0.00,0.00,0.32,109.89
3,Axis Bank Apps,2022-04-01,7.19,1213.44,56.84,43.71,0.00,0.00,0.00,0.00,64.03,1257.15
4,Bajaj Finserv,2022-04-01,0.52,38.61,0.00,0.00,0.00,0.00,0.00,0.00,0.52,38.61
...,...,...,...,...,...,...,...,...,...,...,...,...
2900,Freo #,2025-05-01,0.04,2.27,0.00,0.00,0.00,0.00,0.00,0.00,0.04,2.27
2901,Payworld,2025-05-01,0.03,2.21,0.00,0.00,0.00,0.00,0.00,0.00,0.03,2.21
2902,Punjab Sind Bank Apps,2025-05-01,0.02,8.87,0.00,0.00,0.00,0.00,0.00,0.00,0.02,8.87
2903,Bank of Maharashtra Apps,2025-05-01,0.02,7.44,0.00,0.00,0.00,0.00,0.00,0.00,0.02,7.44


In [254]:
upi_apps.rename(columns={
    "customer_initiated_transactions(volume_in_mn)": "customer_transaction_volume",
    "b2c_transactions(volume_in_mn)":"b2c_transaction_volume",
    "b2b_transactions(_volumne_in_mn)":"b2b_transaction_volume",
    "on-us_transactions(volume_in_mn)":"on_us_transaction_volume",
    "total(volume_in_mn)":"total_volume"
}, inplace=True)


In [255]:
upi_apps.rename(columns={
    "customer_initiated_transactions(value_in_cr)": "customer_transaction_value",
    "b2c_transactions(value_in_cr)":"b2c_transaction_value",
    "b2b_transactions(_value_in_cr)":"b2b_transaction_value",
    "on-us_transactions(value_in_cr)":"on_us_transaction_value",
    "total(value_in_cr)":"total_value"
}, inplace=True)

In [256]:
upi_apps

,application_name,date,customer_transaction_volume,customer_transaction_value,b2c_transaction_volume,b2c_transaction_value,b2b_transaction_volume,b2b_transaction_value,on_us_transaction_volume,on_us_transaction_value,total_volume,total_value
0,Airtel Payments Bank Apps,2022-04-01,5.29,454.64,0.00,0.00,0.00,0.00,0.01,6.26,5.30,460.90
1,Amazon Pay,2022-04-01,73.21,6699.57,0.00,0.00,0.00,0.00,0.00,0.00,73.21,6699.57
2,AU Small Finance Bank Apps,2022-04-01,0.32,109.89,0.00,0.00,0.00,0.00,0.00,0.00,0.32,109.89
3,Axis Bank Apps,2022-04-01,7.19,1213.44,56.84,43.71,0.00,0.00,0.00,0.00,64.03,1257.15
4,Bajaj Finserv,2022-04-01,0.52,38.61,0.00,0.00,0.00,0.00,0.00,0.00,0.52,38.61
...,...,...,...,...,...,...,...,...,...,...,...,...
2900,Freo #,2025-05-01,0.04,2.27,0.00,0.00,0.00,0.00,0.00,0.00,0.04,2.27
2901,Payworld,2025-05-01,0.03,2.21,0.00,0.00,0.00,0.00,0.00,0.00,0.03,2.21
2902,Punjab Sind Bank Apps,2025-05-01,0.02,8.87,0.00,0.00,0.00,0.00,0.00,0.00,0.02,8.87
2903,Bank of Maharashtra Apps,2025-05-01,0.02,7.44,0.00,0.00,0.00,0.00,0.00,0.00,0.02,7.44


In [262]:
upi_apps[["customer_transaction_volume","b2c_transaction_volume",
          "b2b_transaction_volume","on_us_transaction_volume","total_volume"]]=upi_apps[["customer_transaction_volume","b2c_transaction_volume",
          "b2b_transaction_volume","on_us_transaction_volume","total_volume"]].apply(volume_to_num)

In [263]:
upi_apps

,application_name,date,customer_transaction_volume,customer_transaction_value,b2c_transaction_volume,b2c_transaction_value,b2b_transaction_volume,b2b_transaction_value,on_us_transaction_volume,on_us_transaction_value,total_volume,total_value
0,Airtel Payments Bank Apps,2022-04-01,5290000.0,454.64,0.0,0.00,0.0,0.00,10000.0,6.26,5300000.0,460.90
1,Amazon Pay,2022-04-01,73210000.0,6699.57,0.0,0.00,0.0,0.00,0.0,0.00,73210000.0,6699.57
2,AU Small Finance Bank Apps,2022-04-01,320000.0,109.89,0.0,0.00,0.0,0.00,0.0,0.00,320000.0,109.89
3,Axis Bank Apps,2022-04-01,7190000.0,1213.44,56840000.0,43.71,0.0,0.00,0.0,0.00,64030000.0,1257.15
4,Bajaj Finserv,2022-04-01,520000.0,38.61,0.0,0.00,0.0,0.00,0.0,0.00,520000.0,38.61
...,...,...,...,...,...,...,...,...,...,...,...,...
2900,Freo #,2025-05-01,40000.0,2.27,0.0,0.00,0.0,0.00,0.0,0.00,40000.0,2.27
2901,Payworld,2025-05-01,30000.0,2.21,0.0,0.00,0.0,0.00,0.0,0.00,30000.0,2.21
2902,Punjab Sind Bank Apps,2025-05-01,20000.0,8.87,0.0,0.00,0.0,0.00,0.0,0.00,20000.0,8.87
2903,Bank of Maharashtra Apps,2025-05-01,20000.0,7.44,0.0,0.00,0.0,0.00,0.0,0.00,20000.0,7.44


In [264]:
upi_apps[["customer_transaction_value","b2c_transaction_value",
          "b2b_transaction_value","on_us_transaction_value","total_value"]]=upi_apps[["customer_transaction_value","b2c_transaction_value",
          "b2b_transaction_value","on_us_transaction_value","total_value"]].apply(lambda x: x*10000000)

In [265]:
upi_apps

,application_name,date,customer_transaction_volume,customer_transaction_value,b2c_transaction_volume,b2c_transaction_value,b2b_transaction_volume,b2b_transaction_value,on_us_transaction_volume,on_us_transaction_value,total_volume,total_value
0,Airtel Payments Bank Apps,2022-04-01,5290000.0,4.546400e+09,0.0,0.0,0.0,0.0,10000.0,62600000.0,5300000.0,4.609000e+09
1,Amazon Pay,2022-04-01,73210000.0,6.699570e+10,0.0,0.0,0.0,0.0,0.0,0.0,73210000.0,6.699570e+10
2,AU Small Finance Bank Apps,2022-04-01,320000.0,1.098900e+09,0.0,0.0,0.0,0.0,0.0,0.0,320000.0,1.098900e+09
3,Axis Bank Apps,2022-04-01,7190000.0,1.213440e+10,56840000.0,437100000.0,0.0,0.0,0.0,0.0,64030000.0,1.257150e+10
4,Bajaj Finserv,2022-04-01,520000.0,3.861000e+08,0.0,0.0,0.0,0.0,0.0,0.0,520000.0,3.861000e+08
...,...,...,...,...,...,...,...,...,...,...,...,...
2900,Freo #,2025-05-01,40000.0,2.270000e+07,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,2.270000e+07
2901,Payworld,2025-05-01,30000.0,2.210000e+07,0.0,0.0,0.0,0.0,0.0,0.0,30000.0,2.210000e+07
2902,Punjab Sind Bank Apps,2025-05-01,20000.0,8.870000e+07,0.0,0.0,0.0,0.0,0.0,0.0,20000.0,8.870000e+07
2903,Bank of Maharashtra Apps,2025-05-01,20000.0,7.440000e+07,0.0,0.0,0.0,0.0,0.0,0.0,20000.0,7.440000e+07


In [266]:
upi_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2905 entries, 0 to 2904
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   application_name             2905 non-null   object        
 1   date                         2905 non-null   datetime64[ns]
 2   customer_transaction_volume  2905 non-null   float64       
 3   customer_transaction_value   2905 non-null   float64       
 4   b2c_transaction_volume       2904 non-null   float64       
 5   b2c_transaction_value        2904 non-null   float64       
 6   b2b_transaction_volume       2905 non-null   float64       
 7   b2b_transaction_value        2905 non-null   float64       
 8   on_us_transaction_volume     2769 non-null   float64       
 9   on_us_transaction_value      2769 non-null   float64       
 10  total_volume                 2740 non-null   float64       
 11  total_value                  2740 non-null 

In [288]:
upi_apps.rename(columns={"date":"month"},inplace=True)

### CHARGEBACK cleaning

In [271]:
chargeback.head()

,Code,Beneficiary Bank,Total Txns during the month,CB Ratio,Chargebacks Received during the month,Re-presentment Raised during the month,Chargebacks Accepted during the month,month
0,AAC,Akhand Anand Co-op Bank Ltd,12334,0.00000,0.0,0.0,0.0,2022-04-01
1,ACB,ABHYUDAYA CO-OP. BANK LTD,1843899,0.00002,37.0,19.0,18.0,2022-04-01
2,ACU,The Adarsh Co-operative Urban Bank Ltd.,276753,0.00000,0.0,0.0,0.0,2022-04-01
3,ADB,Adarsh Co-operative Bank Limited,84570,0.00001,1.0,0.0,1.0,2022-04-01
4,AGU,The Agrasen Co-Operative Urban Bank Ltd,3098,0.00000,0.0,0.0,0.0,2022-04-01


In [272]:
chargeback.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18922 entries, 0 to 18921
Data columns (total 8 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   Code                                    18922 non-null  object        
 1   Beneficiary Bank                        18922 non-null  object        
 2   Total Txns during the month             18922 non-null  int64         
 3   CB Ratio                                18922 non-null  float64       
 4   Chargebacks Received during the month   18489 non-null  float64       
 5   Re-presentment Raised during the month  18769 non-null  float64       
 6   Chargebacks Accepted during the month   18007 non-null  float64       
 7   month                                   18922 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 1.2+ MB


In [278]:
chargeback.columns=chargeback.columns.str.lower().str.replace(" ","_")

In [284]:
chargeback.rename(columns={"beneficiary_bank":"bank",
                  "total_txns_during_the_month":"total_txns",
                   "chargebacks_received_during_the_month":"chargeback_recieved",
                   "re-presentment_raised_during_the_month":"representment_raised",
                   "chargebacks_accepted_during_the_month":"chargeback_accepted"},inplace=True)

In [285]:
chargeback

,code,bank,total_txns,cb_ratio,chargeback_recieved,representment_raised,chargeback_accepted,month
0,AAC,Akhand Anand Co-op Bank Ltd,12334,0.00000,0.0,0.0,0.0,2022-04-01
1,ACB,ABHYUDAYA CO-OP. BANK LTD,1843899,0.00002,37.0,19.0,18.0,2022-04-01
2,ACU,The Adarsh Co-operative Urban Bank Ltd.,276753,0.00000,0.0,0.0,0.0,2022-04-01
3,ADB,Adarsh Co-operative Bank Limited,84570,0.00001,1.0,0.0,1.0,2022-04-01
4,AGU,The Agrasen Co-Operative Urban Bank Ltd,3098,0.00000,0.0,0.0,0.0,2022-04-01
...,...,...,...,...,...,...,...,...
18917,BMI,BANK OF MAURITIUS,1,0.00000,0.0,0.0,0.0,2025-05-01
18918,PYI,Paytm Payments Bank Limited PYI,1,0.00000,0.0,0.0,0.0,2025-05-01
18919,GPI,GIPL (Slice),1,0.00000,0.0,0.0,0.0,2025-05-01
18920,HCM,HDFC Bank Credit Line HCM,1,0.00000,0.0,0.0,0.0,2025-05-01


### p2p cleaning

In [286]:
p2p.head()

,Month,Total(volumne in mn),total(value in cr),P2P(volume in mn),P2P(value in cr),P2M(volume in mn),P2M(value in cr)
0,2022-04-01,5583.05,983302.27,3319.85,805925.89,2263.20,177376.39
1,2022-08-01,6579.63,1072792.68,3289.26,833370.86,3290.37,239421.82
2,2022-12-01,7829.49,1282055.01,3602.19,991748.31,4227.30,290306.69
3,2022-02-01,4527.49,826843.00,2631.65,663841.98,1895.84,163001.03
4,2022-01-01,4617.15,831993.11,2757.61,667270.65,1859.54,164722.46


In [289]:
p2p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Month                 39 non-null     datetime64[ns]
 1   Total(volumne in mn)  39 non-null     float64       
 2   total(value in cr)    39 non-null     float64       
 3   P2P(volume in mn)     39 non-null     float64       
 4   P2P(value in cr)      39 non-null     float64       
 5   P2M(volume in mn)     39 non-null     float64       
 6   P2M(value in cr)      39 non-null     float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 2.3 KB


In [290]:
p2p.columns=p2p.columns.str.lower().str.replace(" ","_")

In [292]:
p2p.head()

,month,total(volumne_in_mn),total(value_in_cr),p2p(volume_in_mn),p2p(value_in_cr),p2m(volume_in_mn),p2m(value_in_cr)
0,2022-04-01,5583.05,983302.27,3319.85,805925.89,2263.20,177376.39
1,2022-08-01,6579.63,1072792.68,3289.26,833370.86,3290.37,239421.82
2,2022-12-01,7829.49,1282055.01,3602.19,991748.31,4227.30,290306.69
3,2022-02-01,4527.49,826843.00,2631.65,663841.98,1895.84,163001.03
4,2022-01-01,4617.15,831993.11,2757.61,667270.65,1859.54,164722.46


In [294]:
p2p.rename(columns={"total(volumne_in_mn)":"total_volume",
                    "total(value_in_cr)":"total_value",
                    "p2p(volume_in_mn)":"p2p_volume",
                    "p2p(value_in_cr)":"p2p_value",
                    "p2m(volume_in_mn)":"p2m_volume",
                    "p2m(value_in_cr)":"p2m_value"},inplace=True)

In [296]:
p2p.head()

,month,total_volume,total_value,p2p_volume,p2p_value,p2m_volume,p2m_value
0,2022-04-01,5583.05,983302.27,3319.85,805925.89,2263.20,177376.39
1,2022-08-01,6579.63,1072792.68,3289.26,833370.86,3290.37,239421.82
2,2022-12-01,7829.49,1282055.01,3602.19,991748.31,4227.30,290306.69
3,2022-02-01,4527.49,826843.00,2631.65,663841.98,1895.84,163001.03
4,2022-01-01,4617.15,831993.11,2757.61,667270.65,1859.54,164722.46


In [297]:
p2p[["total_volume","p2p_volume","p2m_volume"]]=p2p[["total_volume","p2p_volume","p2m_volume"]].apply(lambda x:x*1000000)

In [298]:
p2p[["total_value","p2p_value","p2m_value"]]=p2p[["total_value","p2p_value","p2m_value"]].apply(lambda x:x*10000000)

In [300]:
p2p.head()

,month,total_volume,total_value,p2p_volume,p2p_value,p2m_volume,p2m_value
0,2022-04-01,5.583050e+09,9.833023e+12,3.319850e+09,8.059259e+12,2.263200e+09,1.773764e+12
1,2022-08-01,6.579630e+09,1.072793e+13,3.289260e+09,8.333709e+12,3.290370e+09,2.394218e+12
2,2022-12-01,7.829490e+09,1.282055e+13,3.602190e+09,9.917483e+12,4.227300e+09,2.903067e+12
3,2022-02-01,4.527490e+09,8.268430e+12,2.631650e+09,6.638420e+12,1.895840e+09,1.630010e+12
4,2022-01-01,4.617150e+09,8.319931e+12,2.757610e+09,6.672706e+12,1.859540e+09,1.647225e+12


In [24]:
#making a df for data modelling

In [50]:
year=pd.DataFrame({"month":prod_stats["month"].unique()})
year

,month
0,2025-06-01
1,2025-05-01
2,2025-04-01
3,2025-03-01
4,2025-02-01
...,...
106,2016-08-01
107,2016-07-01
108,2016-06-01
109,2016-05-01


### importing all data to excel files

In [53]:
#upi_apps.to_excel("C:\\Users\\shrut\\OneDrive\\Desktop\\upi_apps.xlsx", index=False)
#chargeback.to_excel("C:\\Users\\shrut\\OneDrive\\Desktop\\chargeback.xlsx", index=False)
#p2p.to_excel("C:\\Users\\shrut\\OneDrive\\Desktop\\p2p.xlsx", index=False)
#year.to_excel("C:\\Users\\shrut\\OneDrive\\Desktop\\month.xlsx", index=False)

In [52]:
year.to_excel("C:\\Users\\shrut\\OneDrive\\Desktop\\month.xlsx", index=False)